In [1]:
import pandas as pd
import numpy as np
import gc
import re
from sklearn.cluster import KMeans, MeanShift

In [2]:
city_replace = [
    ['peter|stpete|spb', 'SANKT-PETERBURG'],
    ['moscow|moskva|mosocw|moskow', 'MOSCOW'],
    ['novosib|nvsibr', 'NOVOSIBIRSK'],
    ['kater', 'EKATERINBURG'],
    ['n.*novg', 'NIZHNIY NOV'],
    ['novg', 'VEL.NOVGOROD'],
    ['erep', 'CHEREPOVETS'],
    ['rasnod', 'KRASNODAR'],
    ['rasno[yj]', 'KRASNOYARSK'],
    ['sama', 'SAMARA'],
    ['kazan', 'KAZAN'],
    ['soch[iy]', 'SOCHI'],
    ['r[yj]aza', 'RYAZAN'],
    ['arza', 'ARZAMAS'],
    ['podol.?sk', 'PODOLSK'],
    ['himki', 'KHIMKI'],
    ['rostov', 'ROSTOV'], # will ovveride Rostov-Na-Don later
    ['rostov.*do', 'ROSTOV-NA-DON'],
    ['ufa', 'UFA'],
    ['^orel|ory[oe]l', 'OREL'],
    ['korol', 'KOROLEV'],
    ['vkar', 'SYKTYVKAR'],
    ['rozavo|rzavo', 'PETROZAVODSK'],
    ['c.*abinsk', 'CHELYABINSK'],
    ['g omsk|^omsk', 'OMSK'],
    ['tomsk', 'TOMSK'],
    ['vorone', 'VORONEZH'],
    ['[yj]arosl', 'YAROSLAVL'],
    ['novoros', 'NOVOROSSIYSK'],
    ['m[yie]t[yi]s', 'MYTISHCHI'],
    ['kal..?ga', 'KALUGA'],
    ['perm', 'PERM'],
    ['volgog|volgrd', 'VOLGOGRAD'],
    ['kirov[^a-z]|kirov$', 'KIROV'],
    ['krasnogo', 'KRASNOGORSK'],
    ['^mo\W+$|^mo$', 'MO'],
    ['irk', 'IRKUTSK'],
    ['balashi', 'BALASHIKHA'],
    ['kaliningrad', 'KALININGRAD'],
    ['anap', 'ANAPA'],
    ['surgut', 'SURGUT'],
    ['odin[tc]', 'ODINTSOVO'],
    ['kemer', 'KEMEROVO'],
    ['t[yuio].?men', 'TYUMEN'],
    ['sarat', 'SARATOV'],
    ['t[uoy]u?la', 'TULA'],
    ['bert', 'LYUBERTSY'],
    ['kotel', 'KOTELNIKI'],
    ['lipet', 'LIPETSK'],
    ['leznodor', 'ZHELEZNODOROZ'],
    ['domod', 'DOMODEDOVO'],
    ['br[yji][a]nsk|braynsk', 'BRYANSK'],
    ['saransk', 'SARANSK'],
    ['znogor', 'ZHELEZNOGORSK'],
    ['smol', 'SMOLENSK'],
    ['sevolo', 'VSEVOLOZHSK'],
    ['p[uy].*kino', 'PUSHKINO'],
    ['re..?tov', 'REUTOV'],
    ['kursk|koursk', 'KURSK'],
    ['belgorod', 'BELGOROD'],
    ['r[yj]azan', 'RYAZAN'],
    ['solnechno', 'SOLNECHNOGORS'],
    ['utorovsk', 'YALUTOROVSK'],
    ['tver', 'TVER'],
    ['barn', 'BARNAUL'],
    ['to.?l..?.?tt[iy]', 'TOLYATTI'],
    ['i[zjg].?evsk', 'IZHEVSK']
]

In [3]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

#### <font color = 'blue'> Считываем данные</font>

In [4]:
df = pd.read_csv('data/test_set.csv', sep =',')

C:\Users\m1ckyro5aololo\Anaconda2\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head(5)

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date
0,2.211818,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,411",2-1 TOLMACHEVA STRIVANTEEVKA141280 RUSRUS,55.967487,37.913682,ff0476dae4b098a7b16aabe93d4268df,2017-08-24
1,1.331379,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"4,111","SOVETSKIJ, 32IVANTEEVKA141282 RUSRUS",55.971296,37.905184,7cfd9a60282459d4692ecc85b856072e,2017-08-12
2,2.608004,NaN,NaN,NaN,PUSHKINO,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,411","105,KRASOARMEYSKOE SHPUSHKINO141206 RUSRUS",56.016591,37.909098,7e5a532f0029861d8a9c4f0479b9450b,2017-06-17
3,1.916752,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,411","G. IVANTEEVKA, UL.TOLMACHEVA, D.6IVANTEEVKA141...",55.964507,37.937912,2afe7d1bc61b86c449f413bdf2119032,2017-08-12
4,1.981067,NaN,NaN,NaN,MOSCOW,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,"5,814",5 KOMSOMOLSKAYA SQMOSCOW101000 RUSRUS,55.776802,37.657352,ab4f00601ff1d949afc59ee3f804c79c,2017-04-26


#### <font color = 'blue'> Удаляем транзакции без даты</font>

In [6]:
df.drop(df[df['transaction_date'].isnull()].index, axis = 0, inplace = True)

####  <font color = 'blue'> Приводим названия городов к более менее стандартным</font>

In [7]:
len(list(df.city.unique()))

7445

In [8]:
df['city'] = df['city'].str.strip()

In [9]:
df['city'].value_counts().head()

MOSCOW           266704
SANKT-PETERBU    158346
MOSKVA           147708
NOVOSIBIRSK       24219
EKATERINBURG      23998
Name: city, dtype: int64

In [10]:
for city in city_replace :
    #print(city[0], city[1])
    #df['city'] = df['city'].apply(lambda x : re.sub(city[0], city[1], str(x).lower())).str.strip()
    df['city'] = df['city'].apply(lambda x : city[1] if re.search(city[0], str(x).lower()) else x)

In [11]:
df['city'].value_counts().head()

MOSCOW             447091
SANKT-PETERBURG    257498
NOVOSIBIRSK         26806
NIZHNIY NOV         25698
EKATERINBURG        25209
Name: city, dtype: int64

#### <font color = 'blue'>Объединим данные геолокации банкоматов и пос-терминалов (не самый лучший код)</font>

In [12]:
def f_atm(x) :
    if np.isnan(x['atm_address_lat']) or np.isnan(x['atm_address_lon']) :
        return 1
    else :
        return 0
def f_pos(x) :
    if np.isnan(x['pos_address_lat']) or np.isnan(x['pos_address_lon']) :
        return 1
    else :
        return 0

In [13]:
def f_lat(x) :
    if x['pos'] == 0 :
        return x['pos_address_lat'] 
    else :
        return x['atm_address_lat'] 
def f_lon(x) :
    if x['pos'] == 0 :
        return x['pos_address_lon'] 
    else :
        return x['atm_address_lon']

In [14]:
df['atm'] = df.apply(f_atm, axis = 1)
df['pos'] = df.apply(f_pos, axis = 1)
df['lat'] = df.apply(f_lat, axis = 1)
df['lon'] = df.apply(f_lon, axis = 1)
df['lat'] = df.apply(f_lat, axis = 1)
df['lon'] = df.apply(f_lon, axis = 1)

In [15]:
df = df[~df['lat'].isnull()]

#### <font color = 'blue'> Зачастую наблюдаем ситуацию когда у клиентов транзакции проводятся в разных городах. Поэтому у каждого клиента оставим только те города в которых совершено наибольшее число транзакций </font>

In [16]:
df['count'] = 1

In [17]:
df[['customer_id', 'city', 'count']].groupby(['customer_id', 'city'], as_index = False).count().head(10)

,customer_id,city,count
0,00021683ccb416637fe9a4cd35e4606e,NOVOSIBIRSK,78
1,0002d0f8a642272b41c292c12ab6e602,ESSENTOUKI,2
2,0002d0f8a642272b41c292c12ab6e602,ESSENTUKI,19
3,0002d0f8a642272b41c292c12ab6e602,ESSENTUKSKAYA,32
4,0002d0f8a642272b41c292c12ab6e602,PREDGORNYIIY,24
5,0004d182d9fede3ba2534b2d5e5ad27e,SOCHI,120
6,0008c2445518c9392cb356c5c3db3392,SARATOV,22
7,0008c2445518c9392cb356c5c3db3392,UST-KURDYUM,1
8,0008c2445518c9392cb356c5c3db3392,ZONALNYY,1
9,000b373cc4969c0be8e0933c08da67e1,BAZARNYE MATA,2


In [18]:
df_customer_city = df.groupby(['customer_id', 'city'], as_index = False)['count'].count()

In [19]:
map_customer_city = {}
for cust in df_customer_city['customer_id'].unique() :
    tmp = df_customer_city[df_customer_city['customer_id'] == cust].sort_values('count', ascending = False).iloc[0]['city']
    map_customer_city[cust] =  tmp

In [20]:
map_customer_city

{'00021683ccb416637fe9a4cd35e4606e': 'NOVOSIBIRSK',
 '0002d0f8a642272b41c292c12ab6e602': 'ESSENTUKSKAYA',
 '0004d182d9fede3ba2534b2d5e5ad27e': 'SOCHI',
 '0008c2445518c9392cb356c5c3db3392': 'SARATOV',
 '000b373cc4969c0be8e0933c08da67e1': 'DZERZHINSK',
 '000c589e94c95984721de4b2bfb9ee4e': 'NOVOIVANOVSKO',
 '001611e3ac051a0ec91c88bbd9dbeb5a': 'IVANOVO',
 '001691ae3885e80add35148a01e75206': 'SANKT-PETERBURG',
 '002044159304738ea7e3598131809851': 'TOLYATTI',
 '0026319faa345a573522f0a04f5c55bb': 'RYAZAN',
 '0029d9be3692701efe66741fa74a8f8b': 'PODOLSK',
 '002c40ec938e91de248400dec824bd49': 'MOSCOW',
 '00317c648bc11161417b342ad480e724': 'SANKT-PETERBURG',
 '0038ea686d27899b0942409157d04ff2': 'SURGUT',
 '003bc1334379d480c7e5f28240dc40d9': 'MAGNITOGORSK',
 '00450ac1c22c9ee6dda590ff5366236c': 'VORONEZH',
 '00509465377a24375b276c5da9a67fa5': 'SANKT-PETERBURG',
 '005b206d0ffec59e249e6f7adc1b2e83': 'MOSCOW',
 '0069673252af134ecf00854d7ecaed5d': 'CHEREPOVETS',
 '007c458cded5ba3b7b02fcfe8724e80b': 'UF

In [21]:
def filter_city(cust, city, map_customer_city) :
    if map_customer_city[cust] == city :
        return 1
    else :
        return 0

In [22]:
df['filter_city'] = df.apply(lambda x : filter_city(x['customer_id'], x['city'], map_customer_city), axis = 1)

In [23]:
df = df[df['filter_city'] == 1]

#### <font color = 'blue'> По каждому клиенту делаем кластеризацию MeanShift() и берем 2 самых больших класса. Дальше нам надо как-то раскидать их на дом и работу. Если считать, что больше транзакций совершается около дома, то скор будет ~0.24, если же около работы, то скор ~0.26

In [24]:
%%time
lst = []
for cust in log_progress(df['customer_id'].unique(), every = 1) :
    df_tmp = df[df['customer_id'] == cust]
    #print(df_tmp)
    if df_tmp.shape[0] < 2 :
        lst.append([cust, df_tmp['lat'].values[0], df_tmp['lon'].values[0], df_tmp['lat'].values[0], df_tmp['lon'].values[0]])
        continue
    ms = MeanShift(bandwidth = 0.005, cluster_all = False)
    ms.fit(df_tmp[['lat', 'lon']])
    lst.append([cust, df_tmp[ms.labels_ == 0]['lat'].mean(), df_tmp[ms.labels_ == 0]['lon'].mean(),
                          df_tmp[ms.labels_ == 1]['lat'].mean(), df_tmp[ms.labels_ == 1]['lon'].mean()])

VBox(children=(HTML(value=''), IntProgress(value=0, max=9997)))

Wall time: 23min 44s


In [25]:
pd.DataFrame(lst).rename(columns = {0 : '_ID_', 1 : '_WORK_LAT_', 2 : '_WORK_LON_', 
                                  3 : '_HOME_LAT_', 4 : '_HOME_LON_'}).fillna(0).to_csv('res.csv', index = False)

#### <font color = 'blue'> Можно улучшить если по уму раскидывать результаты кластеризации на дом и работу, используя например mcc</font>